<a href="https://colab.research.google.com/github/cagBRT/timeSeries/blob/main/11g_Multiple_Parallel_Series.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Multiple Parallel Series**

Use this model when there are multiple parallel time series and you must predict a value for each.

For example:<br>
For this time series<br>
[[10,15,25]<br>
[20,25,45]<br>
[30,35,65]<br>
[40,45,85]<br>
[50,55,105]<br>
[60,65,125]<br>
[70,75,145]<br>
[80,85,165]<br>
[90,95,185]<br>
<br>
Input:<br>
10,15,25<br>
20,25,45<br>
30,35,65<br><br>
Output:<br>
40,445,85



In [ ]:
from numpy import array
from numpy import hstack
from keras.models import Sequential
from keras.layers import LSTM
from keras.layers import Dense

In [ ]:
# split a multivariate sequence into samples
def split_sequences(sequences, n_steps):
  X, y = list(), list()
  for i in range(len(sequences)):
    # find the end of this pattern
    end_ix = i + n_steps
    # check if we are beyond the dataset
    if end_ix > len(sequences)-1:
     break
    # gather input and output parts of the pattern
    seq_x, seq_y = sequences[i:end_ix, :], sequences[end_ix, :]
    X.append(seq_x)
    y.append(seq_y)
  return array(X), array(y)

In [ ]:
# define input sequence
in_seq1 = array([10, 20, 30, 40, 50, 60, 70, 80, 90])
in_seq2 = array([15, 25, 35, 45, 55, 65, 75, 85, 95])
out_seq = array([in_seq1[i]-(in_seq2[i]*2) for i in range(len(in_seq1))])

In [ ]:
# convert to [rows, columns] structure
in_seq1 = in_seq1.reshape((len(in_seq1), 1))
in_seq2 = in_seq2.reshape((len(in_seq2), 1))
out_seq = out_seq.reshape((len(out_seq), 1))

In [ ]:
dataset = hstack((in_seq1, in_seq2, out_seq))

In [ ]:
# choose a number of time steps
n_steps = 3
# convert into input/output
X, y = split_sequences(dataset, n_steps)
print(X.shape, y.shape)

In [ ]:
# summarize the data
for i in range(len(X)):
  print(X[i], y[i])

In [ ]:
n_features = X.shape[2]

In [ ]:
model = Sequential()
model.add(LSTM(100, activation='relu', return_sequences=True, input_shape=(n_steps,
n_features)))
model.add(LSTM(100, activation='relu')) 
model.add(Dense(n_features)) 
model.compile(optimizer='adam', loss='mse')

In [ ]:
# fit model
model.fit(X, y, epochs=400, verbose=0)

In [ ]:
# demonstrate prediction
x_input = array([[70,75,-80], [80,85,-90], [90,95,-100]]) 
x_input = x_input.reshape((1, n_steps, n_features))
yhat = model.predict(x_input, verbose=0)
print(yhat)